## Load the data set


In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

c:\Users\dinuk\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train, test =  tfds.load("oxford_iiit_pet",split=["train","test"],as_supervised=True)

## Model

In [3]:
base_model=tf.keras.applications.ResNet50(include_top=False,weights="imagenet",input_shape=(224,224,3),pooling="avg")

In [4]:
inputs=tf.keras.layers.Input(shape=(None,None,3),name="input")
resizing=tf.keras.layers.Resizing(224,224)(inputs)
preprocessed=tf.keras.applications.resnet.preprocess_input(resizing)
embeddings=base_model(preprocessed)
output=tf.keras.layers.Flatten()(embeddings)
model=tf.keras.Model(inputs=inputs,outputs=output)


In [5]:
train_ds=train.map(lambda x,y:(tf.expand_dims(x,axis=0),y))
# train_ds=train.batch(1)
train_ds=train_ds.map(lambda x,y:(model(x),y))

test_ds=test.map(lambda x,y:(tf.expand_dims(x,axis=0),y))
# test_ds=test.batch(1) 
test_ds=test_ds.map(lambda x,y:(model(x),y))


In [6]:
train_x=[]
train_y=[]
for x,y in train_ds:
    train_x.append(x[0])
    train_y.append(y)

In [9]:
test_x=[]
test_y=[]
for x,y in test_ds:
    test_x.append(x[0])
    test_y.append(y)
    

In [8]:
from sklearn.neighbors import KNeighborsClassifier

classifer=KNeighborsClassifier(n_neighbors=10,weights="distance",algorithm="auto")
classifer.fit(train_x,train_y)

KNeighborsClassifier(n_neighbors=10, weights='distance')

In [10]:
classifer.score(test_x,test_y)  

0.8383755791768874